In [8]:
import pandas as pd
import requests
import time
import json
import datetime
import timedelta

/tmp/ipykernel_142187/3977951339.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [ ]:
weather_df = pd.read_csv("./jupyter/2024_forecast.csv")

In [ ]:
del weather_df['Unnamed: 0']

In [ ]:
weather_df.head(1)

In [ ]:
weather_df['date'] = ""

In [ ]:
start_date = '2024-1-2'
end_date = '2024-1-2'
params = '{"elems":[{"name":"maxt","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_100"},{"reduce":"pct_ge_90"},{"reduce":"pct_ge_50"},{"reduce":"pct_le_32"}]},{"name":"mint","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_70"},{"reduce":"pct_ge_50"},{"reduce":"pct_le_32"},{"reduce":"pct_le_0"}]},{"name":"pcpn","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_0.01"},{"reduce":"pct_ge_0.10"},{"reduce":"pct_ge_0.50"},{"reduce":"pct_ge_1.0"}]},{"name":"snow","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_0.1"},{"reduce":"pct_ge_1.0"},{"reduce":"pct_ge_3.0"},{"reduce":"pct_ge_6.0"}]},{"name":"snwd","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_1"},{"reduce":"pct_ge_3"},{"reduce":"pct_ge_6"},{"reduce":"pct_ge_12"}]}],"sid":"TOPthr 9","sDate":"{start_date}","eDate":"{end_date}"}'
params = params.replace('{start_date}', start_date)
params = params.replace('{end_date}', end_date)
param_obj = json.loads(params)
url = 'https://data.rcc-acis.org/StnData'
req = requests.post(url, data={'params': params, 'output': 'json'})

In [ ]:
date, max_temp, min_temp, precip, snow, snow_depth = req.json()['data'][0]
day_data = {'date': date, 'actual_max_temp': max_temp, "actual_min_temp": min_temp,
            "actual_precip": precip, "actual_snow": snow, "actual_snow_depth": snow_depth}
ephemera_url = "http://ira.local:5000/api/weather/add"
requests.post(ephemera_url, json=day_data).text

In [ ]:
actuals_df = pd.DataFrame([day_data])

In [ ]:
actuals_df

In [ ]:
for i, r in weather_df.iterrows():
    current_date = f"2024-{r['month']:02d}-{r['day']:02d}"
    weather_df.loc[i, 'date'] = current_date

In [ ]:
day_df_json = pd.merge(weather_df, actuals_df,
                       how='inner').to_json(orient='records')
pd.merge(weather_df, actuals_df,
         how='inner')

In [ ]:
ephemera_url = "http://ira.local:5000/api/weather"
requests.get(ephemera_url).json()

In [ ]:
start_date = datetime.strftime(
    datetime.now() - timedelta(1), '%Y-%m-%d')  # '2024-1-2'
end_date = datetime.strftime(
    datetime.now() - timedelta(1), '%Y-%m-%d')  # '2024-1-2'
params = '{"elems":[{"name":"maxt","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_100"},{"reduce":"pct_ge_90"},{"reduce":"pct_ge_50"},{"reduce":"pct_le_32"}]},{"name":"mint","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_70"},{"reduce":"pct_ge_50"},{"reduce":"pct_le_32"},{"reduce":"pct_le_0"}]},{"name":"pcpn","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_0.01"},{"reduce":"pct_ge_0.10"},{"reduce":"pct_ge_0.50"},{"reduce":"pct_ge_1.0"}]},{"name":"snow","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_0.1"},{"reduce":"pct_ge_1.0"},{"reduce":"pct_ge_3.0"},{"reduce":"pct_ge_6.0"}]},{"name":"snwd","interval":[1,0,0],"duration":1,"smry":[{"reduce":"max"},{"reduce":"mean"},{"reduce":"min"},{"reduce":"pct_ge_1"},{"reduce":"pct_ge_3"},{"reduce":"pct_ge_6"},{"reduce":"pct_ge_12"}]}],"sid":"TOPthr 9","sDate":"{start_date}","eDate":"{end_date}"}'
params = params.replace('{start_date}', start_date)
params = params.replace('{end_date}', end_date)
param_obj = json.loads(params)
url = 'https://data.rcc-acis.org/StnData'
req = requests.post(url, data={'params': params, 'output': 'json'})

date, max_temp, min_temp, precip, snow, snow_depth = req.json()['data'][0]
day_data = {'date': date, 'actual_max_temp': max_temp, "actual_min_temp": min_temp,
            "actual_precip": precip, "actual_snow": snow, "actual_snow_depth": snow_depth}

In [ ]:
ephemera_url = "http://ira.local:5000/api/weather/add"
requests.post(ephemera_url, json=day_data).text